In [132]:
import altair as alt
import pandas as pd
import altair_viewer
alt.data_transformers.disable_max_rows()
altair_viewer._global_viewer._use_bundled_js = False
alt.data_transformers.enable('data_server')

Measure = ['Quantity','Profit','Discount','Sales']

def filterDate(Dimension,typ): #Date inly

    df[Dimension] = pd.to_datetime(df[Dimension],format='%d/%m/%Y')

    if typ == 'year':
        s = str(Dimension+' year')
        df[s] = df[Dimension].dt.year
        return df[s]
    elif typ == 'month':
        s = str(Dimension+' month')
        df[s] = df[Dimension].dt.month
        return df[s]
    elif typ == 'day':
        s = str(Dimension+' day')
        df[s] = df[Dimension].dt.day
        return df[s]

def rangeScale(Di,Meas,df):
    fil = Meas[1]
    if fil == 'average':
        fil = 'mean'

    if len(Di) == 1:
        if (type(Di[0]) == type(['list'])):
            x = str(Di[0][0]+' '+Di[0][1])
        else:
            x = Di[0]
        #print([x])
        tmax = df.groupby([x], as_index=False)[Meas[0]].agg(fil).max()[1]
        tmin = df.groupby([x], as_index=False)[Meas[0]].agg(fil).min()[1]
        
        if tmin > 0:
            tmin = 0
        elif tmax < 0 :
            tmax = 0
        return [tmin,tmax]

    elif (type(Di[0]) == type(['list'])) and (type(Di[1]) == type(['list'])):
        x = str(Di[1][0]+' '+Di[1][1])
        col = str(Di[0][0]+' '+Di[0][1])
    elif type(Di[0]) == type(['list']):
        x = Di[1]
        col = str(Di[0][0]+' '+Di[0][1])
    elif type(Di[1]) == type(['list']):
        x = str(Di[1][0]+' '+Di[1][1])
        col = Di[0]
    else:
        x = Di[1]
        col = Di[0]
    #print(col,x,Meas[0])
    tmax = df.groupby([col,x], as_index=False)[Meas[0]].agg(fil).max()[2]
    tmin = df.groupby([col,x], as_index=False)[Meas[0]].agg(fil).min()[2]
    
    if tmin > 0:
        tmin = 0
    elif tmax < 0 :
        tmax = 0
    
    return [tmin,tmax]

def functionRC(row,column,scale,meas):
    lr = []
    lc = []

    for r in row:
        if type(r) == type(['list']):
            if r[0] in Measure and scale != {} and meas[0] in list(scale.keys()):                         #!!!!!!!!!!!!!!!
                s = r[0]
                lr.append(s)
            else:
                s = str(r[1]+'('+r[0]+')')
                lr.append(s)
        else:
            lr.append(r)

    for c in column:
        if type(c) == type(['list']):
            if c[0] in Measure and scale != {} and meas[0] in list(scale.keys()):                         #!!!!!!!!!!!!!!!!!
                s = c[0]
                lc.append(s)
            else:
                s = str(c[1]+'('+c[0]+')')
                lc.append(s)
        else:
            lc.append(c)
    
    return [lr,lc]

def newdf(df,di,meas,scale):
    if scale == {} or meas[0] not in list(scale.keys()):
        return df
    scmin = scale[meas[0]][0]
    scmax = scale[meas[0]][1]
    dff = df.groupby(di,as_index=False).agg(meas[1])
    dfff = dff.loc[dff[meas[0]] >= scmin]
    dfff = dfff.loc[dfff[meas[0]] <= scmax]
    return dfff


def plotBar(row,column,meas,di,mes,scale): #edit and add dimen in 3 dimen
    #print(row,column,mes)
    print(di,meas)
    if meas[1] == 'average':
        meas = [meas[0],'mean']
    dff = newdf(df,di,meas,scale)
    l = functionRC(row,column,scale,meas)
    lr = l[0]
    lc = l[1]
    if len(lr) == 2 and len(lc) == 0:           # 1 dimension and Measurement on row
        c = alt.Chart(dff).mark_bar(clip=True).encode(
            y= alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
            tooltip = [lr[-2],lr[-1]]
        ).facet(row=lr[-2]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c
    
    elif len(lr) == 0 and len(lc) == 2:         # 1 dimension and Measurement on column
        c = alt.Chart(dff).mark_bar(clip=True).encode(
            x= alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
            tooltip = [lc[-2],lc[-1]]
        ).facet(column=lc[-2]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c
    
    elif len(lr) == 1 and len(lc) == 1:         #1 dimension and Measurement with row and column
        c = alt.Chart(dff).mark_bar(clip=True).encode(
            x=lc[-1],
            y=lr[-1],
            tooltip = [lc[-1],lr[-1]]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c
        
    elif len(lr) == 2 and len(lc) == 1:             
        if mes == 'row':
            c = alt.Chart(dff).mark_bar(clip=True).encode(
                x=lc[-1],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                tooltip = [lr[-2],lr[-1],lc[-1]]
            ).facet(row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(dff).mark_bar(clip=True).encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                y=lr[-1],
                tooltip = [lr[-2],lr[-1],lc[-1]]
            ).facet(row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

    elif len(lr) == 1 and len(lc) == 2:     #dimension2 date error
                 
        if mes == 'row':   
            c = alt.Chart(dff).mark_bar(clip=True).encode(
                x=lc[-1],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                tooltip = [lc[-2],lr[-1],lc[-1]]
            ).facet(column=lc[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(dff).mark_bar(clip=True).encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                y=lr[-1],
                tooltip = [lc[-2],lr[-1],lc[-1]]
            ).facet(column=lc[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

    elif len(lr) == 2 and len(lc) == 2:             ###################
        if mes == 'row':
            c = alt.Chart(dff).mark_bar(clip=True).encode(
                x=lc[-1],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                tooltip = [lc[-2],lr[-1],lc[-2],lc[-1]]
            ).facet(column=lc[-2],row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(dff).mark_bar(clip=True).encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                y=lr[-1],
                tooltip = [lc[-2],lr[-1],lc[-2],lc[-1]]
            ).facet(column=lc[-2],row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c


    elif len(lr) == 1 and len(lc) == 3:
        if mes == 'row':                
            c = alt.Chart(dff).mark_bar(clip=True).encode(        #all Dimen on Col or Row
                x=lc[-2],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                color = lc[-1],
                tooltip = [lc[-3],lc[-2],lc[-1],lr[-1]]
            ).facet(column=lc[-3]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            if column[2][0] in Measure:
                print('x=',lc[-2],'y=',lr[-1],'color=',lc[-1],'col=',lc[-3])
                c = alt.Chart(dff).mark_bar(clip=True).encode(
                    x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                    y=lr[-1],
                    color = lc[-2],
                    tooltip = [lc[-3],lc[-2],lc[-1],lr[-1]]
                ).facet(column=lc[-3]
                ).resolve_scale(x = 'independent',y = 'independent')
                return c
                
            else:
                c = alt.Chart(dff).mark_bar(clip=True).encode(
                    x=alt.X(lc[-2],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                    y=lr[-1],
                    color = lc[-1],
                    tooltip = [lc[-3],lc[-2],lc[-1],lr[-1]]
                ).facet(column=lc[-3]
                ).resolve_scale(x = 'independent',y = 'independent')
                return c

    elif len(lr) == 3 and len(lc) == 1:
        if mes == 'row':
            if row[2][0] in Measure:
                c = alt.Chart(dff).mark_bar(clip=True).encode(
                    x=lc[-1],
                    y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                    color = lr[-2],
                    tooltip = [lc[-1],lr[-3],lr[-2],lr[-1]]
                ).facet(row=lr[-3]
                ).resolve_scale(x = 'independent',y = 'independent')
                return c
            else:
                c = alt.Chart(dff).mark_bar(clip=True).encode(
                    x=lc[-1],
                    y=alt.Y(lr[-2],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                    color = lr[-1],
                    tooltip = [lc[-1],lr[-3],lr[-2],lr[-1]]
                ).facet(row=lr[-3]
                ).resolve_scale(x = 'independent',y = 'independent')
                return c
        else:
            c = alt.Chart(dff).mark_bar(clip=True).encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                y=lr[-2],
                color = lr[-1],
                tooltip = [lc[-1],lr[-3],lr[-2],lr[-1]]
            ).facet(row=lr[-3]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

    elif len(lr) == 3 and len(lc) == 2:
        #print('x=',lc[-1],'y=',lr[-1],'color=',lr[-2],'row=',lr[-3],'column=',lc[-2])
        c = alt.Chart(dff).mark_bar(clip=True).encode(
            x=lc[-1],
            y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
            color = lr[-2],
            tooltip = [lc[-1],lr[-3],lr[-2],lr[-1]]
        ).facet(row=lr[-3] , column = lc[-2]
        ).resolve_scale(y = 'independent')
        return c

    elif len(lr) == 2 and len(lc) == 3: 
        c = alt.Chart(dff).mark_bar(clip=True).encode(
            x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
            y=lr[-1],
            color = lc[-2],
            tooltip = [lc[-3],lc[-2],lc[-1],lr[-1]]
        ).facet(column=lc[-3] , row = lr[-2]
        ).resolve_scale(x = 'independent')
        return c
       
def plotLine(row,column,meas,di,mes,scale):

    print(di,meas)
    if meas[1] == 'average':
        meas = [meas[0],'mean']
    dff = newdf(df,di,meas,scale)
    l = functionRC(row,column,scale,meas)
    lr = l[0]
    lc = l[1]

    if len(lr) == 1 and len(lc) == 1:         #1 dimension and Measurement with row and column
        c = alt.Chart(dff).mark_line(point=True).encode(
            x=alt.X(lc[-1]),
            y=alt.Y(lr[-1]),
            tooltip = [lc[-1],lr[-1]]
        ).resolve_legend(size='independent')
        return c
        
    elif len(lr) == 2 and len(lc) == 1:             
        if mes == 'row':
            c = alt.Chart(dff).mark_line(point=True).encode(
                x=lc[-1],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                tooltip = [lc[-1],lr[-1]]
            ).facet(row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(dff).mark_line(point=True).encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                y=lr[-1],
                tooltip = [lr[-1],lc[-1]]
            ).facet(row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

    elif len(lr) == 1 and len(lc) == 2:     #dimension2 date error
        if mes == 'row':   
            c = alt.Chart(dff).mark_line(point=True).encode(
                x=lc[-1],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                tooltip = [lc[-1],lr[-1]]
            ).facet(column=lc[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(dff).mark_line(point=True).encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                y=lr[-1],
                tooltip = [lr[-1],lc[-1]]
            ).facet(column=lc[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

    elif len(lr) == 2 and len(lc) == 2:             ###################
        if mes == 'row':
            c = alt.Chart(dff).mark_line(point=True).encode(
                x=lc[-1],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                tooltip = [lc[-1],lr[-1]]
            ).facet(column=lc[-2],row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(dff).mark_line(point=True).encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas,dff))),
                y=lr[-1],
                tooltip = [lr[-1],lc[-1]]
            ).facet(column=lc[-2],row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

def plotPie(row,column,meas,di,mes,scale):
    if meas[1] == 'average':
        meas = [meas[0],'mean']
    dff = newdf(df,di,meas,scale)
    l = functionRC(row,column,scale,meas)
    lr = l[0]
    lc = l[1]
    
    '''base = alt.Chart(df).mark_arc().encode(
        theta=alt.Theta(str(fil+'('+Mes+'):Q')), 
        color=alt.Color(s, type="nominal"), 
        tooltip = [s,str(fil+'('+Mes+'):Q')]
    ).resolve_scale(theta = 'independent',color = 'independent')'''
    if len(lr) == 1 and len(lc) == 1:
        if mes == 'row':
            M = lr[-1]
            Di = lc
        else:
            M = lc[-1]
            Di = lr
        c = alt.Chart(dff).mark_arc().encode(
            alt.Theta(M), 
            alt.Color(Di[-1], type="nominal"),
            tooltip = [M,Di[-1]]
        ).resolve_scale(theta = 'independent',color = 'independent')
        return c
        
    elif len(lr) == 2 and len(lc) == 1:             
        if mes == 'row':
            c = alt.Chart(dff).mark_arc().encode(
                alt.Theta(lr[-1]), 
                alt.Color(lc[-1], type="nominal"),
                alt.Row(lr[-2]),
                tooltip = [lr[-2],lc[-1],lr[-1]]
            ).resolve_scale(theta = 'independent',color = 'independent')
            return c
        else:
            c = alt.Chart(dff).mark_arc().encode(
                alt.Theta(lc[-1]), 
                alt.Color(lr[-1], type="nominal"),
                alt.Row(lr[-2]),
                tooltip = [lr[-2],lc[-1],lr[-1]]
            ).resolve_scale(theta = 'independent',color = 'independent')
            return c

    elif len(lr) == 1 and len(lc) == 2:     #dimension2 date error
        if mes == 'row':   
            c = alt.Chart(dff).mark_arc().encode(
                alt.Theta(lr[-1]), 
                alt.Color(lc[-1], type="nominal"),
                alt.Column(lc[-2]),
                tooltip = [lc[-2],lc[-1],lr[-1]]
            ).resolve_scale(theta = 'independent',color = 'independent')
            return c
        else:
            c = alt.Chart(dff).mark_arc().encode(
                alt.Theta(lc[-1]), 
                alt.Color(lr[-1], type="nominal"),
                alt.Column(lc[-2]),
                tooltip = [lc[-2],lr[-1],lc[-1]]
            ).resolve_scale(theta = 'independent',color = 'independent')
            return c

    elif len(lr) == 2 and len(lc) == 2:             ###################
        if mes == 'row':
            c = alt.Chart(dff).mark_arc().encode(
                alt.Theta(lr[-1]), 
                alt.Color(lc[-1], type="nominal"),
                alt.Row(lr[-2]),
                alt.Column(lc[-2]),
                tooltip = [lr[-2],lc[-2],lc[-1],lr[-1]]
            ).resolve_scale(theta = 'independent',color = 'independent')
            return c
        else:
            c = alt.Chart(dff).mark_arc().encode(
                alt.Theta(lc[-1]), 
                alt.Color(lr[-1], type="nominal"),
                alt.Row(lr[-2]),
                alt.Column(lc[-2]),
                tooltip = [lr[-2],lc[-2],lr[-1],lc[-1]]
            ).resolve_scale(theta = 'independent',color = 'independent')
            return c

    return c

def plotGraph(row,column,tp,scale): #!!!!
    if tp == 'line':
        mes = 'col'
        for r in row:
            if type(r) == type(['list']):
                if r[0] in Measure:
                    mes = 'row'
        #print(mes)
        if mes == 'row':
            chart = []
            if type(row[0]) == type(['list']):  #Datetime and Meas
                if row[0][0] not in Measure:    #Datetime
                    l = [*column]
                    l.append(row[0])
                    for r in range(len(row)-1):
                        chart.append(plotLine([row[0],row[r+1]],column,row[r+1],l,mes,scale))
                    return alt.vconcat(*chart)
                else:                           #Measure
                    for r in range(len(row)):
                        chart.append(plotLine([row[r]],column,row[r],[*column],mes,scale))
                    return alt.vconcat(*chart)
            else:                               #Dimension
                l = [*column]
                l.append(row[0])
                for r in range(len(row)-1):
                    chart.append(plotLine([row[0],row[r+1]],column,row[r+1],l,mes,scale))
                return alt.vconcat(*chart)
        else:
            chart = []
            if type(column[0]) == type(['list']):
                if column[0][0] not in Measure:
                    l = [*row]
                    l.append(column[0])
                    for c in range(len(column)-1):
                        chart.append(plotLine(row,[column[0],column[c+1]],column[c+1],l,mes,scale))
                    return alt.vconcat(*chart)
                else:
                    for c in range(len(column)):
                        chart.append(plotLine(row,[column[c]],column[c],[*row],mes,scale))
                    return alt.vconcat(*chart)
            else:
                l = [*row]
                l.append(column[0])
                for c in range(len(column)-1):
                    chart.append(plotLine(row,[column[0],column[c+1]],column[c+1],l,mes,scale))
                return alt.vconcat(*chart)

    elif tp == 'bar':
        mes = 'col'
        for r in row:
            if type(r) == type(['list']):
                if r[0] in Measure:
                    mes = 'row'
        #print(mes)
        if mes == 'row':
            chart = []
            if type(row[0]) == type(['list']):  #Datetime and Meas
                if row[0][0] not in Measure:    #Datetime
                    if row[1][0] not in Measure:       #4Dimension (Can't plot)
                        print('4 Dimension')
                        l = [*column]
                        l.append(row[0])
                        l.append(row[1])
                        for r in range(len(row)-2):
                            chart.append(plotBar([row[0],row[1],row[r+2]],column,row[r+2],l,mes,scale))
                        return alt.vconcat(*chart)
                    else:
                        l = [*column]
                        l.append(row[0])
                        for r in range(len(row)-1):
                            chart.append(plotBar([row[0],row[r+1]],column,row[r+1],l,mes,scale))
                        return alt.vconcat(*chart)
                else:                           #Measure
                    for r in range(len(row)):
                        chart.append(plotBar([row[r]],column,row[r],[*column],mes,scale))
                    return alt.vconcat(*chart)
            else:                               #Dimension
                if row[1][0] not in Measure:       #4Dimension
                    l = [*column]
                    l.append(row[0])
                    l.append(row[1])
                    for r in range(len(row)-2):
                        chart.append(plotBar([row[0],row[1],row[r+2]],column,row[r+2],l,mes,scale))
                    return alt.vconcat(*chart)
                else:
                    l = [*column]
                    l.append(row[0])
                    for r in range(len(row)-1):
                        chart.append(plotBar([row[0],row[r+1]],column,row[r+1],l,mes,scale))
                    return alt.vconcat(*chart)
        else:
            chart = []
            if type(column[0]) == type(['list']):
                if column[0][0] not in Measure:
                    if column[1][0] not in Measure:
                        l = [*row]
                        l.append(column[0])
                        l.append(column[1])
                        for c in range(len(column)-2):
                            chart.append(plotBar(row,[column[0],column[1],column[c+2]],column[c+2],l,mes,scale))
                        return alt.vconcat(*chart)
                    else:
                        l = [*row]
                        l.append(column[0])
                        for c in range(len(column)-1):
                            chart.append(plotBar(row,[column[0],column[c+1]],column[c+1],l,mes,scale))
                        return alt.vconcat(*chart)
                else:
                    for c in range(len(column)):
                        chart.append(plotBar(row,[column[c]],column[c],[*row],mes,scale))
                    return alt.vconcat(*chart)
            else:
                if column[1][0] not in Measure:
                    l = [*row]
                    l.append(column[0])
                    l.append(column[1])
                    for c in range(len(column)-2):
                        chart.append(plotBar(row,[column[0],column[1],column[c+2]],column[c+2],l,mes,scale))
                    return alt.vconcat(*chart)
                else:
                    l = [*row]
                    l.append(column[0])
                    for c in range(len(column)-1):
                        chart.append(plotBar(row,[column[0],column[c+1]],column[c+1],l,mes,scale))
                    return alt.vconcat(*chart)

    elif tp == 'pie':
        mes = 'col'
        for r in row:
            if type(r) == type(['list']):
                if r[0] in Measure:
                    mes = 'row'
        #print(mes)
        if mes == 'row':
            chart = []
            if type(row[0]) == type(['list']):  #Datetime and Meas
                if row[0][0] not in Measure:    #Datetime
                    l = [*column]
                    l.append(row[0])
                    for r in range(len(row)-1):
                        chart.append(plotPie([row[0],row[r+1]],column,row[r+1],l,mes,scale))
                    return alt.vconcat(*chart).resolve_scale(theta = 'independent',color = 'independent')
                else:                           #Measure
                    for r in range(len(row)):
                        chart.append(plotPie([row[r]],column,row[r],[*column],mes,scale))
                    return alt.vconcat(*chart).resolve_scale(theta = 'independent',color = 'independent')
            else:                               #Dimension
                l = [*column]
                l.append(row[0])
                for r in range(len(row)-1):
                    chart.append(plotPie([row[0],row[r+1]],column,row[r+1],l,mes,scale))
                return alt.vconcat(*chart).resolve_scale(theta = 'independent',color = 'independent')
        else:
            chart = []
            if type(column[0]) == type(['list']):
                if column[0][0] not in Measure:
                    l = [*row]
                    l.append(column[0])
                    for c in range(len(column)-1):
                        chart.append(plotPie(row,[column[0],column[c+1]],column[c+1],l,mes,scale))
                    return alt.vconcat(*chart).resolve_scale(theta = 'independent',color = 'independent')
                else:
                    for c in range(len(column)):
                        chart.append(plotPie(row,[column[c]],column[c],[*row],mes,scale))
                    return alt.vconcat(*chart).resolve_scale(theta = 'independent',color = 'independent')
            else:
                l = [*row]
                l.append(column[0])
                for c in range(len(column)-1):
                    chart.append(plotPie(row,[column[0],column[c+1]],column[c+1],l,mes,scale))
                return alt.vconcat(*chart).resolve_scale(theta = 'independent',color = 'independent')

df = pd.read_csv('Superstore.csv', encoding='windows-1252')

for d in ['Order Date','Ship Date']:
    filterDate(d,'year')
    filterDate(d,'month')
    filterDate(d,'day')

D1 = ['Order Date','month']
D2 = ['Ship Date','month']

D3 = 'Category'
D4 = 'Sub-Category'

D5 = 'Country/Region'
D6 = 'Region'
D7 = 'State'
D8 = 'City'

D9 = 'Ship Mode'
D10 = 'Segment'

M1 = ['Quantity','sum']
M2 = ['Discount','sum']
M3 = ['Profit','max']
M4 = ['Sales','sum']

column = [D7]
row = [M3]
scale = {'Profit': [0, 1000]} #can not with Dimension Datetime
#scale = {}
#column , row = row ,column

#plotBar(row,column,f1)
#plotLine(row,column,f1)
#plotPie(row,column,f1)
plotGraph(row,column,'pie',scale)
#print(list(scale.keys()))
#newdf(df,[D9],M1,scale)

alt.VConcatChart(...)

In [ ]:
Measure = ['Quantity','Profit','Discount','Sales']
row = [M2,D1,M3,D5,D7]
column = [D6,D3,M1]
r = 0
c = 0
for i in row:
    if type(i) == type(['list']):
        if i[0] not in Measure:
            r += 1
    elif i not in Measure:
        r +=1

for j in column:
    if type(j) == type(['list']):
        if j[0] not in Measure:
            c += 1
    elif j not in Measure:
        c +=1
print(r,c)

In [188]:
df = pd.read_csv('Superstore.csv', encoding='windows-1252')
for d in ['Order Date','Ship Date']:
    filterDate(d,'year')
    filterDate(d,'month')
    filterDate(d,'day')

#dff = df.groupby(['Category','Order Date year'], as_index=False).agg('sum')
dff = df.groupby([pd.Grouper(key = 'Order Date' , freq='Y'),'Category'],as_index=False).agg('sum')
#dff = dff.groupby([pd.Grouper(freq='Y'),'Category']).agg('sum')
#dfff = dff.loc[dff['Segment'] < 60000]
dfff = dff.loc[dff['Profit'] < 30000]
dfff = dfff.loc[dfff['Profit'] > 0]
dff

Row ID  Postal Code        Sales  Quantity  \
Order Date Category                                                        
2017-12-31 Furniture         2043994   24203915.0  157192.8531      1623   
           Office Supplies   6156744   68289344.0  151776.4120      4569   
           Technology        1703277   20777988.0  175278.2330      1389   
2018-12-31 Furniture         2197339   24860701.0  170518.2370      1775   
           Office Supplies   6171109   64789483.0  137233.4630      4715   
           Technology        2045248   21547247.0  162780.8090      1489   
2019-12-31 Furniture         2921431   30976941.0  198901.4360      2193   
           Office Supplies   7561354   84480704.0  183939.9820      5946   
           Technology        2296019   25524143.0  226364.1800      1698   
2020-12-31 Furniture         3530562   38143653.0  215387.2692      2437   
           Office Supplies  10121328  113180893.0  246097.1750      7676   
           Technology        3196610   34738152.0  271730.8110      2363   

                            Discount      Profit  Order Date year  \
Order Date Category                                                 
2017-12-31 Furniture           76.66   5457.7255           849157   
           Office Supplies    190.30  22593.4161          2454689   
           Technology          48.50  21492.8325           716035   
2018-12-31 Furniture           76.29   3015.2029           912136   
           Office Supplies    198.90  25099.5338          2504338   
           Technology          51.90  33503.8670           825362   
2019-12-31 Furniture           99.42   6959.9531          1134678   
           Office Supplies    238.20  35061.2292          3161754   
           Technology          62.70  39773.9920           926721   
2020-12-31 Furniture          116.52   3018.3913          1385720   
           Office Supplies    320.40  39736.6217          4044040   
           Technology          81.30  50684.2566          1260480   

                            Order Date month  Order Date day  Ship Date year  \
Order Date Category                                                            
2017-12-31 Furniture                    3354            6701          849168   
           Office Supplies              9456           19078         2454716   
           Technology                   2853            5401          716050   
2018-12-31 Furniture                    3610            7010          912138   
           Office Supplies              9831           19798         2504357   
           Technology                   3262            6391          825365   
2019-12-31 Furniture                    4397            8516         1134685   
           Office Supplies             12194           23918         3161776   
           Technology                   3488            6874          926725   
2020-12-31 Furniture                    5431           10867         1385732   
           Office Supplies             15393           30590         4044067   
           Technology                   4781            9447         1260483   

                            Ship Date month  Ship Date day  
Order Date Category                                         
2017-12-31 Furniture                   3275           6742  
           Office Supplies             9276          19571  
           Technology                  2721           5347  
2018-12-31 Furniture                   3633           7387  
           Office Supplies             9760          20020  
           Technology                  3271           6642  
2019-12-31 Furniture                   4376           8816  
           Office Supplies            12099          24918  
           Technology                  3492           7088  
2020-12-31 Furniture                   5379          10678  
           Office Supplies            15297          31556  
           Technology                  4806           9977

In [141]:
c = alt.Chart(dfff).mark_line(point=True).encode(
    y = 'year(Order Date)',
    x= 'Profit',
    #tooltip = ['Segment','Profit']
).facet(row='Category'
).resolve_scale(x='independent',y='independent')
c


ValueError: Order Date encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

alt.FacetChart(...)

In [13]:
for d in ['Order Date','Ship Date']:
    x = filterDate(d,'year')
list(x.drop_duplicates())

[2019, 2018, 2017, 2020, 2021]